<h2>解析库(XPath BeautifulSoup PyQuery)的使用</h2>

提取页面信息时使用正则表达式比较烦琐，写错可能导致匹配失败（使用正则表达式提取页面信息不方便）。

对于网页的节点来说，它可以定义 id、class 或其他属性。而且节点之间还有层次关系，

在网页中可以通过 XPath 或 CSS 选择器来定位一个或多个节点。

实现这个操作可以使用的解析库非常多，其中比较强大的库有 lxml、Beautiful Soup、pyquery 等。

<h3>使用 XPath</h3>

(XPath, XML Path Language)，即 XML 路径语言，它是一门在 XML 文档中查找信息的语言。它最初是用来搜寻 XML 文档的，但是它同样适用于 HTML 文档的搜索。


XPath 于 1999 年 11 月 16 日成为 W3C 标准，它被设计为供 XSLT、XPointer 以及其他 XML 解析软件使用，更多的文档可以访问其官方网站：[https://www.w3.org/TR/xpath/](https://www.w3.org/TR/xpath/)

**XPath 常用规则**

```
表　达　式	描　　述
nodename	选取此节点的所有子节点
/	从当前节点选取直接子节点
//	从当前节点选取子孙节点
.	选取当前节点
..	选取当前节点的父节点
@	选取属性
```
XPath 的常用匹配规则，示例：`//title[@lang='eng']`，它代表选择所有名称为 title，同时属性 lang 的值为 eng 的节点。

通过实例来感受一下使用 XPath 来对网页进行解析的过程，相关代码如下：
> 注意：HTML 文本中的最后一个 `li` 节点是没有闭合的，但是 etree 模块可以自动修正 HTML 文本。（并且还自动添加了 `body`、`html` 节点）
> 
> 调用 `tostring` 方法即可输出修正后的 HTML 代码，但是结果是 `bytes` 类型。这里利用 `decode` 方法将其转成 `str` 类型

In [1]:
from lxml import etree
text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text)
result = etree.tostring(html)
print(result.decode('utf-8'))

<html><body><div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </li></ul>
 </div>
</body></html>


另外，也可以直接读取文本文件进行解析，示例如下：
```python
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = etree.tostring(html)
print(result.decode('utf-8'))
```
> 会多出一个 `DOCTYPE` 的声明

**所有节点**

一般会用 `//` 开头的 XPath 规则来选取所有符合要求的节点。这里以前面的 HTML 文本为例，如果要选取所有节点，可以这样实现：
```python
from lxml import etree
html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//*')
print(result)
```

使用 `*` 代表匹配所有节点，也就是整个 HTML 文本中的所有节点都会被获取。

返回形式是一个列表，每个元素是 Element 类型，其后跟了节点的名称，如 `html、body、div、ul、li、a` 等，所有节点都包含在列表中了。

想获取所有 li 节点：`result = html.xpath('//li')`

**子节点**

通过 `/` 或 `//` 即可查找元素的子节点或子孙节点。

假如想选择 li 节点的所有直接 a 子节点，可以这样实现：

In [2]:
from lxml import etree

text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text)
result = html.xpath('//li/a')  # 等价于 `//li//a`
print(result)

[<Element a at 0x1519a62c1c0>, <Element a at 0x1519a62c280>, <Element a at 0x1519a62c440>, <Element a at 0x1519a62c900>, <Element a at 0x1519b844600>]


**父节点**

假如知道了子节点，可以用`..`来查找父节点呢。

比如，现在首先选中 `href` 属性为 `link4.html` 的 `a` 节点，然后再获取其父节点，然后再获取其 `class` 属性，

相关代码：`result = html.xpath('//a[@href="link4.html"]/../@class')`

也可以通过 `parent::` 来获取父节点，代码：`result = html.xpath('//a[@href="link4.html"]/parent::*/@class')`

**文本获取**

用 XPath 中的 `text` 方法获取节点中的文本，尝试获取前面 li 节点中的文本，相关代码：
`result = html.xpath('//li[@class="item-0"]/text()')` 或者 `result = html.xpath('//li[@class="item-0"]//text()')`

**属性获取**

节点属性用 `@`符号就可以获取。例如，想获取所有 li 节点下所有 a 节点的 href 属性，代码：`result = html.xpath('//li/a/@href')`

**属性多值匹配**

有时候，某些节点的某个属性可能有多个值，例如：
> 这里 HTML 文本中 li 节点的 class 属性有两个值 li 和 li-first，此时如果还想用之前的属性匹配获取，就无法匹配了，此时的运行结果 —— `[]`

In [3]:
from lxml import etree  
text = '''  
<li class="li li-first"><a href="link.html">first item</a></li> 
'''  
html = etree.HTML(text)  
result = html.xpath('//li[@class="li"]/a/text()')  
print(result)

[]


这时就需要用 `contains` 方法了，代码可以改写如下：

In [4]:
from lxml import etree  
text = '''  
<li class="li li-first"><a href="link.html">first item</a></li>  
'''  
html = etree.HTML(text)  
result = html.xpath('//li[contains(@class, "li")]/a/text()')  
print(result)

['first item']


**多属性匹配**

根据多个属性确定一个节点，需要同时匹配多个属性。此时可以使用运算符 `and` 来连接，示例如下：

In [5]:
from lxml import etree  
text = '''  
<li class="li li-first" name="item"><a href="link.html">first item</a></li>
'''  
html = etree.HTML(text)  
result = html.xpath('//li[contains(@class, "li") and @name="item"]/a/text()')  
print(result)

['first item']


运算符及其介绍
```
运算符	描　　述	实　　例	返　回　值
or	或	age=19 or age=20	如果 age 是 19，则返回 true。如果 age 是 21，则返回 false
and	与	age>19 and age<21	如果 age 是 20，则返回 true。如果 age 是 18，则返回 false
mod	计算除法的余数	5 mod 2	1
计算两个节点集	//book
+	加法	6 + 4	10
-	减法	6 - 4	2
*	乘法	6 * 4	24
div	除法	8 div 4	2
=	等于	age=19	如果 age 是 19，则返回 true。如果 age 是 20，则返回 false
!=	不等于	age!=19	如果 age 是 18，则返回 true。如果 age 是 19，则返回 false
<	小于	age<19	如果 age 是 18，则返回 true。如果 age 是 19，则返回 false
<=	小于或等于	age<=19	如果 age 是 19，则返回 true。如果 age 是 20，则返回 false
>	大于	age>19	如果 age 是 20，则返回 true。如果 age 是 19，则返回 false
>=	大于或等于	age>=19	如果 age 是 19，则返回 true。如果 age 是 18，则返回 false
```

**按序选择**

在选择的时候某些属性可能同时匹配了多个节点，但是只想要其中的某个节点，如第二个节点或者最后一个节点，可以利用**中括号传入索引的方法获取特定次序的节点**，示例如下：

In [6]:
from lxml import etree

text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text)
result = html.xpath('//li[1]/a/text()')
print(result)
result = html.xpath('//li[last()]/a/text()')
print(result)
result = html.xpath('//li[position()<3]/a/text()')
print(result)
result = html.xpath('//li[last()-2]/a/text()')
print(result)

['first item']
['fifth item']
['first item', 'second item']
['third item']


**节点轴选择**

XPath 提供了很多节点轴选择方法，包括获取子元素、兄弟元素、父元素、祖先元素等，示例如下：

In [7]:
from lxml import etree

text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html"><span>first item</span></a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text)
result = html.xpath('//li[1]/ancestor::*')
print(result)
result = html.xpath('//li[1]/ancestor::div')
print(result)
result = html.xpath('//li[1]/attribute::*')
print(result)
result = html.xpath('//li[1]/child::a[@href="link1.html"]')
print(result)
result = html.xpath('//li[1]/descendant::span')
print(result)
result = html.xpath('//li[1]/following::*[2]')
print(result)
result = html.xpath('//li[1]/following-sibling::*')
print(result)

[<Element html at 0x15198c30280>, <Element body at 0x15198c2b2c0>, <Element div at 0x15198c30200>, <Element ul at 0x15198c30540>]
[<Element div at 0x15198c30200>]
['item-0']
[<Element a at 0x15198c306c0>]
[<Element span at 0x15198c30200>]
[<Element a at 0x15198c304c0>]
[<Element li at 0x15198c305c0>, <Element li at 0x15198c30400>, <Element li at 0x15198c30380>, <Element li at 0x15198c30680>]


<h3>使用 Beautiful Soup</h3>